In [1]:
import pygame
import math

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# 2. Constantes do Jogo (Cores, Tamanhos, Velocidades, etc.)
# Cores (RGB)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0) # Cor do Pac-Man
# ... (outras cores para fantasmas, etc.)

# Configurações do Labirinto
CELL_SIZE = 25 # Tamanho de cada célula (bloco) em pixels
WALL_LINE_THICKNESS = 4
# ... (outras constantes de tamanho se necessário)

# Configurações do Pac-Man
PACMAN_RADIUS = int(CELL_SIZE * 0.4)
PACMAN_SPEED = 2 # Velocidade em pixels por frame

# Configurações da Pílula de Poder
POWER_PILL_DURATION = 8 * 1000 # Duração do efeito da pílula de poder em milissegundos (8 segundos)
POWER_PILL_SCORE = 50 # Pontos ganhos por comer uma pílula de poder

# Configurações dos Fantasmas (opcional, pode ser adicionado depois)
GHOST_SPEED = 1
# ...

In [3]:
# --- 3. Definição do Labirinto (Matriz 2D) ---
# Convenções para a matriz:
# 0 = Caminho vazio (onde o Pac-Man e fantasmas podem andar)
# 1 = Parede (obstáculo)
# 2 = Pílula (pequeno ponto comestível)
# 3 = Pílula de Poder (ponto grande, deixa fantasmas vulneráveis)
# 4 = Porta da casa dos fantasmas (apenas fantasmas podem atravessar)

level = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 4, 4, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1], # Linha da porta dos fantasmas
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], # Túnel lateral 
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1],
    [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
    [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
# Calcular as dimensões da tela com base no labirinto
SCREEN_WIDTH = len(level[0]) * CELL_SIZE
SCREEN_HEIGHT = len(level) * CELL_SIZE

# ---Contagem de Pílulas e Cópia do Labirinto Original ---
original_level = [row[:] for row in level] 

total_pills = 244
print(f"DEBUG_INICIAL: Total de pílulas contadas no início: {total_pills}") # 244 (240 pílulas normais e 4 pílulas de poder).
# Variável global (ou gerenciada por uma classe de jogo) para as pílulas restantes
pills_left = total_pills # O número de pílulas no início do nível


DEBUG_INICIAL: Total de pílulas contadas no início: 244


In [4]:
# 4. Classes do Jogo (PacMan e Ghost)
# --- Classe PacMan ---
class PacMan:
    def __init__(self, start_x_grid, start_y_grid, radius, speed, cell_size):
        self.grid_x = start_x_grid
        self.grid_y = start_y_grid

        # Variáveis para a pílula de poder
        self.is_powered_up = False
        self.power_up_timer = 0 # Usaremos pygame.time.get_ticks() para gerenciar isso
        
        # Posição em pixels, centralizada na célula
        self.pixel_x = float(self.grid_x * cell_size + cell_size // 2)
        self.pixel_y = float(self.grid_y * cell_size + cell_size // 2)

        self.radius = radius
        self.color = YELLOW
        self.speed = speed
        self.cell_size = cell_size

        # Direção de movimento atual (pixels por frame)
        self.dx = 0
        self.dy = 0
        
        # Direção desejada (para viradas)
        self.desired_dx_val = 0 # Valor de -1, 0 ou 1
        self.desired_dy_val = 0 # Valor de -1, 0 ou 1


        # Animação da boca do Pac-Man
        self.mouth_open = True  # Começa com a boca aberta
        self.mouth_speed = 1.5  # Velocidade da animação da boca (quanto mais rápido, mais rápido abre/fecha)
        self.current_mouth_angle = 45 # Ângulo inicial da boca aberta (metade do ângulo total da boca)
        self.target_mouth_angle = 45 # Ângulo que a boca tenta alcançar
        self.closed_mouth_angle = 0  # Ângulo da boca fechada
        self.open_mouth_angle = 45   # Ângulo da boca totalmente aberta (metade do ângulo total da boca)

        # Direção para o desenho da boca (0=right, 90=up, 180=left, 270=down)
        self.facing_angle = 0 # Começa virado para a direita (0 graus)
        
        self.last_moved_direction = (1, 0) # (dx, dy) para a última direção em que realmente se moveu


    def set_direction(self, dx_val, dy_val):
        """Define a direção desejada do Pac-Man."""
        self.desired_dx_val = dx_val
        self.desired_dy_val = dy_val

    def get_grid_pos(self):
        """Retorna a posição do Pac-Man no grid (inteiros)."""
        return int(self.pixel_x // self.cell_size), int(self.pixel_y // self.cell_size)

    def is_aligned_to_grid(self):
        """Verifica se o Pac-Man está aproximadamente no centro de uma célula."""
        # Margem de erro para alinhamento
        tolerance = self.speed // 2 

        # Calcula a diferença entre a posição pixel e o centro da célula atual
        center_x_of_current_grid = self.grid_x * self.cell_size + self.cell_size // 2
        center_y_of_current_grid = self.grid_y * self.cell_size + self.cell_size // 2
        
        # Verifica se o pixel_x está próximo do centro horizontal da célula
        aligned_x = abs(self.pixel_x - center_x_of_current_grid) <= tolerance
        # Verifica se o pixel_y está próximo do centro vertical da célula
        aligned_y = abs(self.pixel_y - center_y_of_current_grid) <= tolerance

        return aligned_x and aligned_y


    def can_move_in_direction(self, dx_check, dy_check, game_level):
        """
        Verifica se é possível mover para a próxima célula na direção (dx_check, dy_check).
        Considera a próxima célula inteira.
        """
        current_grid_x, current_grid_y = self.get_grid_pos()

        target_grid_x = current_grid_x + dx_check
        target_grid_y = current_grid_y + dy_check


        # --- LÓGICA PARA TÚNEL: Permite movimento para fora do grid para teletransporte ---
        tunnel_row = 15 # <--- Defina esta constante para a linha do túnel (índice 15)

        # Se o Pac-Man está na linha do túnel E está tentando se mover para fora da tela
        if current_grid_y == tunnel_row:
            if (current_grid_x == 0 and dx_check < 0) or \
               (current_grid_x == len(game_level[0]) - 1 and dx_check > 0):
                return True # Permite o movimento "off-screen" para o túnel
    
        # Garante que a próxima célula está dentro dos limites do labirinto
        if not (0 <= target_grid_y < len(game_level) and 0 <= target_grid_x < len(game_level[0])):
            return False # Fora dos limites, não pode mover

        # Verifica se a próxima célula é uma parede (1)
        if game_level[target_grid_y][target_grid_x] == 1:
            return False # É uma parede, não pode mover
        
        # Adicione lógica para a porta dos fantasmas (4)
        # Pac-Man não pode atravessar a porta dos fantasmas
        if game_level[target_grid_y][target_grid_x] == 4:
            return False

        return True # Pode mover

    def update(self, game_level):
        """Atualiza a posição do Pac-Man e lida com colisões, viradas, e animação."""
        
        # 1. Atualizar a posição do Pac-Man no grid (sempre no início do update)
        self.grid_x = int(self.pixel_x // self.cell_size)
        self.grid_y = int(self.pixel_y // self.cell_size)

        points_gained = 0 # Inicializa a pontuação ganha neste frame
        pill_eaten_this_frame = False # Para saber se comeu algo
        
        # --- DEPURAÇÃO CRÍTICA: Posição do Pac-Man (SEMPRE ATIVO) ---
        # print(f"PacMan Grid: ({self.grid_x}, {self.grid_y}) | Pixel: ({self.pixel_x:.2f}, {self.pixel_y:.2f}) | DX: {self.dx}, DY: {self.dy}")

        # 2. Lógica de virada e movimento (com base na desired_direction)
        if self.desired_dx_val != 0 or self.desired_dy_val != 0:
            if self.is_aligned_to_grid():
                if self.can_move_in_direction(self.desired_dx_val, self.desired_dy_val, game_level):
                    self.dx = self.desired_dx_val * self.speed
                    self.dy = self.desired_dy_val * self.speed
                    self.pixel_x = float(self.grid_x * self.cell_size + self.cell_size // 2)
                    self.pixel_y = float(self.grid_y * self.cell_size + self.cell_size // 2)
                else:
                    self.desired_dx_val = 0
                    self.desired_dy_val = 0

        # 3. Aplicar movimento atual e tratar colisão com paredes
        if self.dx != 0 or self.dy != 0:
            if self.can_move_in_direction(self.dx // self.speed, self.dy // self.speed, game_level):
                self.pixel_x += self.dx
                self.pixel_y += self.dy
            else:
                self.dx = 0
                self.dy = 0
                self.pixel_x = float(self.grid_x * self.cell_size + self.cell_size // 2)
                self.pixel_y = float(self.grid_y * self.cell_size + self.cell_size // 2)

            # 4. Lógica de Consumo de Pílulas e Pílulas de Poder (Verifica na célula atual)
            # Re-calcula a posição no grid APÓS o movimento de pixel, para pegar a célula para a qual ele acabou de se mover
            current_grid_x_after_move = int(self.pixel_x // self.cell_size)
            current_grid_y_after_move = int(self.pixel_y // self.cell_size)

            # Garante que a posição está dentro dos limites do labirinto (especialmente útil para túneis)
            if 0 <= current_grid_y_after_move < len(game_level) and 0 <= current_grid_x_after_move < len(game_level[0]):
                cell_content_before_consumption = game_level[current_grid_y_after_move][current_grid_x_after_move] # <--- PEGA O CONTEÚDO ANTES DE MUDAR

                # Se a célula contém uma pílula ou pílula de poder
                if cell_content_before_consumption == 2 or cell_content_before_consumption == 3:
                    # Remove a pílula do labirinto (sempre)
                    game_level[current_grid_y_after_move][current_grid_x_after_move] = 0  
                    
                    # Define que uma pílula foi comida neste frame
                    pill_eaten_this_frame = True 

                    # Atribui os pontos e o estado de poder
                    if cell_content_before_consumption == 2:  # Se for uma pílula normal
                        points_gained = 10
                        
                    elif cell_content_before_consumption == 3:  # Se for uma pílula de poder
                        self.is_powered_up = True
                        self.power_up_timer = pygame.time.get_ticks()
                        points_gained = POWER_PILL_SCORE
                        print(f"Pac-Man Powered Up! Tempo: {self.power_up_timer}")
                    
            # --- Lógica: Passagem pelo Túnel Lateral ---
            tunnel_row = 15 # A linha do grid onde o túnel está
    
            # --- DEPURAÇÃO: Verificando condições do túnel (agora ativadas se grid_y for 15) ---
            if self.grid_y == tunnel_row: # Se Pac-Man está na linha do túnel
                # print(f"Tunnel Check: PacMan na linha {self.grid_y} (esperado {tunnel_row})") # DEPURAÇÃO ADICIONAL
                if self.grid_x == 0 and self.dx < 0: # Se está na coluna 0 e tentando ir para a esquerda
                    self.pixel_x = float((len(game_level[0]) - 1) * self.cell_size + self.cell_size // 2)
                    self.grid_x = len(game_level[0]) - 1
                    print("TELEPORTE: Esquerda para Direita!")
                elif self.grid_x == len(game_level[0]) - 1 and self.dx > 0: # Se está na coluna 27 e tentando ir para a direita
                    self.pixel_x = float(0 * self.cell_size + self.cell_size // 2)
                    self.grid_x = 0
                    print("TELEPORTE: Direita para Esquerda!")
            # FIM DA LÓGICA DO TÚNEL
                    
            # 5. Lógica de Animação da Boca
            self.last_moved_direction = (self.dx // self.speed, self.dy // self.speed) # Normaliza

            if self.mouth_open:
                self.current_mouth_angle -= self.mouth_speed * 2
                if self.current_mouth_angle <= self.closed_mouth_angle:
                    self.mouth_open = False
            else:
                self.current_mouth_angle += self.mouth_speed * 2
                if self.current_mouth_angle >= self.open_mouth_angle:
                    self.mouth_open = True
        else: # Pac-Man está parado
            if self.current_mouth_angle > self.closed_mouth_angle:
                self.current_mouth_angle -= self.mouth_speed * 2
                if self.current_mouth_angle <= self.closed_mouth_angle:
                    self.current_mouth_angle = self.closed_mouth_angle

        # 6. Atualiza o ângulo de rotação da boca (sempre, mesmo parado)
        if self.last_moved_direction == (1, 0): # Direita
            self.facing_angle = 0
        elif self.last_moved_direction == (-1, 0): # Esquerda
            self.facing_angle = 180
        elif self.last_moved_direction == (0, -1): # Cima
            self.facing_angle = 90
        elif self.last_moved_direction == (0, 1): # Baixo
            self.facing_angle = 270

        # 7. Gerenciar o temporizador da Pílula de Poder (sempre, mesmo parado)
        if self.is_powered_up:
            current_time = pygame.time.get_ticks()
            if current_time - self.power_up_timer > POWER_PILL_DURATION:
                self.is_powered_up = False
                self.power_up_timer = 0
                print("Pac-Man Power-Up Acabou!")

        # 8. Retornar os pontos ganhos
        return points_gained, pill_eaten_this_frame

    def draw(self, screen):
        """Desenha o Pac-Man na tela com animação da boca."""
        
        # Desenha o Pac-Man como um círculo (corpo)
        pygame.draw.circle(screen, self.color, (int(self.pixel_x), int(self.pixel_y)), self.radius)

        # Desenha o "buraco" da boca como um triângulo preto
        # (Lembrete: 'import math' deve estar no topo do seu arquivo)
        
        # Ponto 1 (centro do Pac-Man)
        point1 = (int(self.pixel_x), int(self.pixel_y))
        
        # Ponto 2 (borda superior da boca, ajustada pela direção)
        angle_rad_top = math.radians(self.facing_angle + self.current_mouth_angle)
        point2 = (int(self.pixel_x + self.radius * math.cos(angle_rad_top)),
                  int(self.pixel_y - self.radius * math.sin(angle_rad_top))) # -sin porque Y cresce para baixo
        
        # Ponto 3 (borda inferior da boca, ajustada pela direção)
        angle_rad_bottom = math.radians(self.facing_angle - self.current_mouth_angle)
        point3 = (int(self.pixel_x + self.radius * math.cos(angle_rad_bottom)),
                  int(self.pixel_y - self.radius * math.sin(angle_rad_bottom))) # -sin porque Y cresce para baixo
        
        # Desenha o triângulo preto que simula a boca aberta
        pygame.draw.polygon(screen, BLACK, [point1, point2, point3])
    
# (Futuramente, adicione classes para Fantasmas, Pílulas, Power-Ups, etc., aqui)

In [5]:
# 5. Inicialização do Pygame
pygame.init()
pygame.font.init()
game_font = pygame.font.Font(None, 24)
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Pac-Man em Python")
clock = pygame.time.Clock() # Para controlar o FPS

In [6]:
# 6. Criação de Instâncias dos Objetos do Jogo
# Crie seu Pac-Man
pacman = PacMan(1, 3, PACMAN_RADIUS, PACMAN_SPEED, CELL_SIZE)
score = 0
last_score_rendered = -1 # <--- Variável para controlar a renderização do score

# --- Lógica de Reinício de Nível ---
def reset_level():
    global level, pills_left, score, pacman, original_level # <--- Usa 'global' para modificar variáveis globais
    
    # 1. Redefinir o labirinto para o estado original
    level = [row[:] for row in original_level] # Copia o labirinto original de volta
    
    # 2. Redefinir a contagem de pílulas
    pills_left = total_pills 
    
    # 3. Redefinir a posição do Pac-Man
    # (Você pode querer uma posição inicial diferente para cada nível ou sempre a mesma)
    pacman.pixel_x = float(1 * CELL_SIZE + CELL_SIZE // 2) # Posição inicial Pac-Man
    pacman.pixel_y = float(3 * CELL_SIZE + CELL_SIZE // 2)
    pacman.dx = 0
    pacman.dy = 0
    pacman.desired_dx_val = 0
    pacman.desired_dy_val = 0
    pacman.is_powered_up = False
    pacman.power_up_timer = 0
    pacman.mouth_open = True # Reseta a boca para aberta
    pacman.current_mouth_angle = pacman.open_mouth_angle
    pacman.facing_angle = 0
    pacman.last_moved_direction = (1, 0)

    # 4. (Futuramente) Redefinir a posição e o estado dos fantasmas

    print("Nível Reiniciado! Todas as pílulas consumidas.")

In [7]:
# 7. Loop Principal do Jogo
running = True
while running:
    # 7.1. Processamento de Eventos (Entrada do Usuário, Fechar Janela)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False # Encerra o loop principal
        if event.type == pygame.KEYDOWN:
            # Controle do Pac-Man com as setas
            if event.key == pygame.K_LEFT:
                pacman.set_direction(-1, 0)
            elif event.key == pygame.K_RIGHT:
                pacman.set_direction(1, 0)
            elif event.key == pygame.K_UP:
                pacman.set_direction(0, -1)
            elif event.key == pygame.K_DOWN:
                pacman.set_direction(0, 1)

    # 7.2. Atualização do Estado do Jogo (Lógica do Jogo)
    points_gained, pill_eaten_this_frame = pacman.update(level) # Pega os pontos retornados pelo PacMan.update()
    
    if points_gained > 0:
        score += points_gained  # Adiciona ao placar total
        
    if pill_eaten_this_frame: # Se alguma pílula foi comida neste frame
        pills_left -= 1 # Decrementa a contagem de pílulas restantes
        print(f"DEBUG: Pílula comida! Pílulas restantes: {pills_left}")
        
    # --- NOVO: DEPURAÇÃO PARA ENCONTRAR PÍLULAS RESTANTES ---
    current_pills_in_maze = 0
    remaining_pill_coords = [] # Lista para armazenar as coordenadas das pílulas
    for r_idx, row in enumerate(level):
        for c_idx, cell in enumerate(row):
            if cell == 2 or cell == 3:
                current_pills_in_maze += 1
                remaining_pill_coords.append((c_idx, r_idx)) # Adiciona (coluna, linha)
    
    # Imprime informações detalhadas apenas quando há poucas pílulas restantes
    if current_pills_in_maze > 0 and current_pills_in_maze <= 5: # Ajuste o "5" para ver mais ou menos
        print(f"DEBUG_FINAL_PILLS: Pílulas na matriz: {current_pills_in_maze}. Coordenadas: {remaining_pill_coords}")
    
    # Opcional: Se houver uma discrepância entre o contador e o que o maze mostra
    # if pills_left != current_pills_in_maze:
    #    print(f"!!! DISCREPÂNCIA: pills_left = {pills_left} mas maze tem {current_pills_in_maze} pílulas. !!!")

    # FIM DA DEPURAÇÃO DE PÍLULAS RESTANTES 
    # --- Verifica Condição de Reinício do Nível ---
    if pills_left <= 0:
        print("DEBUG: Condição de reinício atingida (pills_left <= 0)!")
        reset_level() # Chama a função para reiniciar o nível
        
    # (Futuramente, atualize fantasmas, verifique colisões Pac-Man/pílula, Pac-Man/fantasma, etc.)

    # 7.3. Desenho na Tela
    screen.fill(BLACK) # Limpa a tela com preto

    # Desenhar o labirinto
    # A lógica de desenho das paredes será mais complexa
    for row_index, row in enumerate(level):
        for col_index, cell in enumerate(row):
            x = col_index * CELL_SIZE
            y = row_index * CELL_SIZE

            # Coordenadas do centro da célula
            center_x = x + CELL_SIZE // 2
            center_y = y + CELL_SIZE // 2

            if cell == 1: # Se a célula é uma parede
                # Desenhar as linhas da parede com base nos vizinhos
                # Cor das paredes
                current_wall_color = BLUE

                # 1. Linhas Horizontais
                # Desenha a linha superior da parede se não houver parede acima
                if row_index > 0 and level[row_index - 1][col_index] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x, y + WALL_LINE_THICKNESS // 2), 
                                     (x + CELL_SIZE, y + WALL_LINE_THICKNESS // 2), 
                                     WALL_LINE_THICKNESS)
                # Desenha a linha inferior da parede se não houver parede abaixo
                if row_index < len(level) - 1 and level[row_index + 1][col_index] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x, y + CELL_SIZE - WALL_LINE_THICKNESS // 2), 
                                     (x + CELL_SIZE, y + CELL_SIZE - WALL_LINE_THICKNESS // 2), 
                                     WALL_LINE_THICKNESS)
                # Desenha linha horizontal se esta parede se conecta a outra parede na mesma linha
                if col_index < len(row) - 1 and level[row_index][col_index + 1] == 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (center_x, center_y), 
                                     (center_x + CELL_SIZE, center_y), 
                                     WALL_LINE_THICKNESS)

                # 2. Linhas Verticais
                # Desenha a linha esquerda da parede se não houver parede à esquerda
                if col_index > 0 and level[row_index][col_index - 1] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x + WALL_LINE_THICKNESS // 2, y), 
                                     (x + WALL_LINE_THICKNESS // 2, y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                # Desenha a linha direita da parede se não houver parede à direita
                if col_index < len(row) - 1 and level[row_index][col_index + 1] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x + CELL_SIZE - WALL_LINE_THICKNESS // 2, y), 
                                     (x + CELL_SIZE - WALL_LINE_THICKNESS // 2, y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                # Desenha linha vertical se esta parede se conecta a outra parede na mesma coluna
                if row_index < len(level) - 1 and level[row_index + 1][col_index] == 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (center_x, center_y), 
                                     (center_x, center_y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                
                # --- Lógica para Cantos e Junções (Simplificado, pode ser melhorado) ---
                # A lógica acima desenha os segmentos retos. Cantos (onde linhas se encontram
                # em L, T, cruz) precisarão de ajustes ou de algoritmos mais robustos
                # para parecerem perfeitos. Para um visual de néon, muitas vezes você desenha
                # círculos nos cantos ou usa técnicas de arredondamento.

                # Desenha um pequeno círculo no centro de cada parede.
                # Isso pode ajudar a "preencher" a junção das linhas e dar um efeito arredondado.
                pygame.draw.circle(screen, current_wall_color, (center_x, center_y), WALL_LINE_THICKNESS // 2)


            elif cell == 2: # Pílula
                pygame.draw.circle(screen, WHITE, (x + CELL_SIZE // 2, y + CELL_SIZE // 2), CELL_SIZE // 4)
            elif cell == 3: # Pílula de Poder
                pygame.draw.circle(screen, WHITE, (x + CELL_SIZE // 2, y + CELL_SIZE // 2), CELL_SIZE // 2)
            elif cell == 4: # Porta da casa dos fantasmas (pode manter como um retângulo fino)
                pygame.draw.rect(screen, (100, 100, 100), (x, y + CELL_SIZE // 2 - 2, CELL_SIZE, 4))

    # Desenhar o Pac-Man
    pacman.draw(screen)

    # --- NOVA LÓGICA: Exibir Pontuação (e futuramente vidas) ---
    if score != last_score_rendered:
        score_text = game_font.render(f"Score: {score}", True, WHITE )  # Renderiza o texto
        last_score_rendered = score

    # Ajuste a posição do texto na tela
    screen.blit(score_text, (5, SCREEN_HEIGHT - 30))  # Exemplo: no canto inferior esquerdo

    # 7.4. Atualizar a Exibição (Mostrar tudo que foi desenhado)
    pygame.display.flip()

    # 7.5. Controle de FPS (Quadros por Segundo)
    clock.tick(60) # Limita o jogo a 60 FPS


DEBUG: Pílula comida! Pílulas restantes: 243
DEBUG: Pílula comida! Pílulas restantes: 242
DEBUG: Pílula comida! Pílulas restantes: 241
DEBUG: Pílula comida! Pílulas restantes: 240
DEBUG: Pílula comida! Pílulas restantes: 239
DEBUG: Pílula comida! Pílulas restantes: 238
DEBUG: Pílula comida! Pílulas restantes: 237
DEBUG: Pílula comida! Pílulas restantes: 236
DEBUG: Pílula comida! Pílulas restantes: 235
DEBUG: Pílula comida! Pílulas restantes: 234
DEBUG: Pílula comida! Pílulas restantes: 233
DEBUG: Pílula comida! Pílulas restantes: 232
DEBUG: Pílula comida! Pílulas restantes: 231
DEBUG: Pílula comida! Pílulas restantes: 230
DEBUG: Pílula comida! Pílulas restantes: 229
DEBUG: Pílula comida! Pílulas restantes: 228
DEBUG: Pílula comida! Pílulas restantes: 227
DEBUG: Pílula comida! Pílulas restantes: 226
DEBUG: Pílula comida! Pílulas restantes: 225
Pac-Man Powered Up! Tempo: 10724
DEBUG: Pílula comida! Pílulas restantes: 224
DEBUG: Pílula comida! Pílulas restantes: 223
DEBUG: Pílula comida! 

In [8]:
# 8. Finalização do Pygame (Após o Loop Principal)
pygame.quit()